Make sure you download the 2016 Household LSMS survey data for Malawi from https://microdata.worldbank.org/index.php/catalog/lsms and put it in `../data/countries/malawi_2016/LSMS`.

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
BASE_DIR = '..'
COUNTRY = 'malawi_2016'
COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')

In [4]:
import sys
sys.path.append(BASE_DIR)
from utils import create_space

In [5]:
'''
The goal of each of a processor function is to output a dataframe with the following columns:
country, cluster_lat, cluster_lon, rCSI, FCS, HDDS

Each row should represent one cluster by combining the household data
'''

def process_malawi():
    lsms_dir = os.path.join(COUNTRIES_DIR, COUNTRY, 'LSMS')
    geovariables_file = 'HouseholdGeovariables_csv/HouseholdGeovariablesIHS4.csv'
    rcsi_file = 'hh_mod_h.csv'
    fcs_hdds_file = 'hh_mod_g2.csv'
    
    lat_col = 'lat_modified'
    lon_col = 'lon_modified'
    
    for file in [geovariables_file, rcsi_file, fcs_hdds_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
        
    # latitude and longitude
    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_geo['cluster_lat'] = df_geo[lat_col]
    df_geo['cluster_lon'] = df_geo[lon_col]
    df_geo = df_geo[['case_id', 'cluster_lat', 'cluster_lon']]
    df_geo.dropna(inplace=True)
    
    # rCSI info
    df_rcsi = pd.read_csv(os.path.join(lsms_dir, 'hh_mod_h.csv'))
    df_rcsi = df_rcsi[['case_id', 'hh_h01', 'hh_h02a', 'hh_h02b', 'hh_h02c', 'hh_h02d', 'hh_h02e']]
    df_rcsi.dropna(inplace=True)
    df_rcsi['rCSI'] = 1*df_rcsi['hh_h02a'] + 1*df_rcsi['hh_h02b'] + \
                        2*df_rcsi['hh_h02c'] + 2*df_rcsi['hh_h02d'] +2*df_rcsi['hh_h02e']
    df_rcsi['rCSI'].loc[df_rcsi['rCSI'] > 42] = 42
    
    # FCSI and HDDS info
    df_fcs_hdds = pd.read_csv(os.path.join(COUNTRIES_DIR, COUNTRY, 'LSMS', 'hh_mod_g2.csv'))
    df_fcs_hdds = df_fcs_hdds[['case_id', 'hh_g08a', 'hh_g08b', 'hh_g08c', 'hh_g08d', 
             'hh_g08e', 'hh_g08f', 'hh_g08g', 'hh_g08h', 'hh_g08i', 'hh_g08j']]
    df_fcs_hdds.dropna(inplace=True)
    # this chooses the max of hh_g08a and hh_g08b per row
    hh_staple = df_fcs_hdds['hh_g08a'].copy()
    idx = df_fcs_hdds['hh_g08a'] < df_fcs_hdds['hh_g08b']
    hh_staple.loc[idx] = df_fcs_hdds['hh_g08b'].loc[idx]
    df_fcs_hdds['FCS'] =  2*hh_staple + 3*df_fcs_hdds['hh_g08c'] + 1*df_fcs_hdds['hh_g08d'] + \
            4*df_fcs_hdds['hh_g08e'] + 1*df_fcs_hdds['hh_g08f'] + 4*df_fcs_hdds['hh_g08g'] + \
            0.5*df_fcs_hdds['hh_g08h'] + 0.5*df_fcs_hdds['hh_g08i']
    hdds_cereals = (hh_staple > 0).astype(int)
    hdds_c = (df_fcs_hdds['hh_g08c'] > 0).astype(int)
    hdds_d = (df_fcs_hdds['hh_g08d'] > 0).astype(int)
    hdds_e = (df_fcs_hdds['hh_g08e'] > 0).astype(int)
    hdds_f = (df_fcs_hdds['hh_g08f'] > 0).astype(int)
    hdds_g = (df_fcs_hdds['hh_g08g'] > 0).astype(int)
    df_fcs_hdds['HDDS'] = hdds_cereals + hdds_c + hdds_d + hdds_e + hdds_f + hdds_g
    df_fcs_hdds = df_fcs_hdds[['case_id', 'FCS', 'HDDS']]
    
    df_combined = pd.merge(df_geo, df_rcsi, on='case_id', how='inner')
    df_combined = pd.merge(df_combined, df_fcs_hdds, on='case_id', how='inner')
    df_combined.drop(['case_id'], axis=1, inplace=True)
    # averages metric per household per cluster
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).mean().reset_index()
    df_clusters['country'] = 'malawi_2016'
    return df_clusters

In [6]:
df_mw = process_malawi()

/opt/conda/envs/predicting-poverty-replication/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
df_mw.shape

(780, 12)

In [8]:
df_mw.head()

,cluster_lat,cluster_lon,hh_h01,hh_h02a,hh_h02b,hh_h02c,hh_h02d,hh_h02e,rCSI,FCS,HDDS,country
0,-17.095150,35.217213,1.0000,4.1250,3.8125,3.5625,1.9375,1.3750,21.6875,33.71875,3.7500,malawi_2016
1,-17.092351,35.114643,1.1875,2.8125,2.6250,2.8750,2.0625,0.6875,16.4375,33.65625,3.5625,malawi_2016
2,-17.016698,35.079629,1.1250,2.1250,2.6250,3.1250,1.3750,1.6250,17.0000,42.21875,4.6250,malawi_2016
3,-16.977243,35.205706,1.1250,4.0625,2.0625,3.1875,1.5000,2.3125,20.1250,34.25000,4.5000,malawi_2016
4,-16.956385,35.168967,1.1875,3.9375,2.2500,2.4375,1.5625,0.0625,14.3125,33.25000,3.8750,malawi_2016


In [9]:
os.makedirs(os.path.join(COUNTRIES_DIR, COUNTRY, 'processed'), exist_ok=True)

In [10]:
df_mw.to_csv(os.path.join(COUNTRIES_DIR, COUNTRY, 'processed', 'clusters.csv'), index=False)